In [ ]:
import schedule
import requests
import zipfile
import yfinance as yf
import MetaTrader5 as mt5
import pandas as pd
import pandas_ta as ta
import numpy as np
import plotly.graph_objects as go
from datetime import datetime, timedelta, time, timezone
from IPython.display import display
import plotly.express as px
import tdclient as TDClient
import dukascopy_python as dp

In [14]:
ACCESS_TOKEN = "9559a0f87d2bb2c55bfcc6ede397c67c-995047624043d66e955900a3ab9495e4"
ACCOUNT_ID = "101-002-37084035-001"
from oandapyV20 import API
import oandapyV20.endpoints.instruments as instruments
import time

client = API(access_token=ACCESS_TOKEN)

def fetch_oanda_data(pair="EUR_USD", start="2005-01-01", end="2025-09-01"):
    start_dt = datetime.fromisoformat(start)
    end_dt = datetime.fromisoformat(end)

    candles = []
    chunk = timedelta(days=3)  # fetch 1 month per request

    while start_dt < end_dt:
        chunk_end = min(start_dt + chunk, end_dt)

        params = {
            "granularity": "M1",
            "price": "M",
            "from": start_dt.strftime("%Y-%m-%dT%H:%M:%SZ"),
            "to": chunk_end.strftime("%Y-%m-%dT%H:%M:%SZ"),
        }

        r = instruments.InstrumentsCandles(instrument=pair, params=params)
        data = client.request(r)

        for c in data["candles"]:
            if c["complete"]:
                candles.append({
                    "time": c["time"],
                    "open": float(c["mid"]["o"]),
                    "high": float(c["mid"]["h"]),
                    "low": float(c["mid"]["l"]),
                    "close": float(c["mid"]["c"]),
                    "volume": c["volume"]
                })

        print(f"Fetched {len(data['candles'])} candles: {start_dt} → {chunk_end}")

        # move forward
        start_dt = chunk_end
        time.sleep(0.2)  # avoid rate limits

    df = pd.DataFrame(candles)
    df["time"] = pd.to_datetime(df["time"])
    return df

# Example: 10 years EUR/USD M1
df = fetch_oanda_data("EUR_USD", "2005-01-01", "2025-09-01")
df.to_csv("EURUSD_OANDA_2005-01_2025-09.csv", index=True)
df


Fetched 1618 candles: 2005-01-01 00:00:00 → 2005-01-04 00:00:00
Fetched 4212 candles: 2005-01-04 00:00:00 → 2005-01-07 00:00:00
Fetched 1469 candles: 2005-01-07 00:00:00 → 2005-01-10 00:00:00
Fetched 4128 candles: 2005-01-10 00:00:00 → 2005-01-13 00:00:00
Fetched 2688 candles: 2005-01-13 00:00:00 → 2005-01-16 00:00:00
Fetched 2913 candles: 2005-01-16 00:00:00 → 2005-01-19 00:00:00
Fetched 3991 candles: 2005-01-19 00:00:00 → 2005-01-22 00:00:00
Fetched 1586 candles: 2005-01-22 00:00:00 → 2005-01-25 00:00:00
Fetched 4165 candles: 2005-01-25 00:00:00 → 2005-01-28 00:00:00
Fetched 1454 candles: 2005-01-28 00:00:00 → 2005-01-31 00:00:00
Fetched 4108 candles: 2005-01-31 00:00:00 → 2005-02-03 00:00:00
Fetched 2647 candles: 2005-02-03 00:00:00 → 2005-02-06 00:00:00
Fetched 3007 candles: 2005-02-06 00:00:00 → 2005-02-09 00:00:00
Fetched 3899 candles: 2005-02-09 00:00:00 → 2005-02-12 00:00:00
Fetched 1549 candles: 2005-02-12 00:00:00 → 2005-02-15 00:00:00
Fetched 4085 candles: 2005-02-15 00:00:0

,time,open,high,low,close,volume
0,2005-01-02 18:29:00+00:00,1.35600,1.35600,1.35600,1.35600,1
1,2005-01-02 18:38:00+00:00,1.35600,1.35600,1.35600,1.35600,1
2,2005-01-02 18:51:00+00:00,1.35670,1.35670,1.35670,1.35670,1
3,2005-01-02 18:52:00+00:00,1.35650,1.35650,1.35650,1.35650,1
4,2005-01-02 18:55:00+00:00,1.35680,1.35680,1.35680,1.35680,1
...,...,...,...,...,...,...
7531705,2025-08-31 23:55:00+00:00,1.16898,1.16898,1.16894,1.16894,26
7531706,2025-08-31 23:56:00+00:00,1.16894,1.16898,1.16894,1.16898,15
7531707,2025-08-31 23:57:00+00:00,1.16898,1.16904,1.16898,1.16902,33
7531708,2025-08-31 23:58:00+00:00,1.16901,1.16904,1.16900,1.16904,25


In [27]:
# Conecte mt5
mt5.initialize()
# logine mt5
login = 1600017186
password = '#XVfl9yb1D5'
server = 'OANDA-Demo-1'

mt5.login(login,password,server)

False

In [ ]:
# this function retreives olhc data from mt5 account and return a data frame
def get_ohlc(symbol, timeframe, start_datetime, end_datetime):
    ohlc = mt5.copy_rates_range(symbol, timeframe, start_datetime, end_datetime)
    df = pd.DataFrame(ohlc)
    df['time'] = pd.to_datetime(df['time'], unit= 's')#.dt.tz_localize('UTC')
    return df

In [ ]:
# 1 minut df
symbol = 'EURUSD'
pos_size = 1
timeframe = mt5.TIMEFRAME_M1
start_datetime = datetime(2025,8,1)
end_datetime = datetime(2025,8,31)
df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
df.tail()

In [ ]:
zip_path = "HISTDATA_COM_MT_EURUSD_M1202509.zip"

with zipfile.ZipFile(zip_path) as z:
    # usually only one file inside
    file_name = z.namelist()[0]  
    with z.open(file_name) as f:
        df_csv = pd.read_csv(f, header=None)
        df_csv = df_csv.rename(columns={
    0: "Date",
    1: "time",
    2: "open",
    3: "high",
    4: "low",
    5: "close",
    6: "volume",})
        df_csv = pd.DataFrame(df_csv)
        df_csv["datetime"] = pd.to_datetime(
    df_csv["Date"] + " " + df_csv["time"], 
    format="%Y.%m.%d %H:%M"
)
df_csv.set_index("datetime", inplace=True)
df_csv.drop(columns=["Date", "time"], inplace=True)

# Current df is in UTC
df_csv.index = df_csv.index.tz_localize("UTC")

# Convert to ICMarkets timezone (UTC+2, or UTC+3 if DST)
df_csv.index = df_csv.index.tz_convert("Etc/GMT-2")   # note: "Etc/GMT-2" means UTC+2


print(df_csv.columns.tolist())
df_may = df_csv.loc["2025-08-01":"2025-08-29 23:56:00"]
df_csv.tail()


In [23]:
import zipfile
import pandas as pd

zip_path = "HISTDATA_COM_MT_EURUSD_M12024.zip"

with zipfile.ZipFile(zip_path) as z:
    # usually only one file inside
    file_name = z.namelist()[0]  
    with z.open(file_name) as f:
        df = pd.read_csv(f, header=None)
        df = df.rename(columns={
    0: "Date",
    1: "time",
    2: "open",
    3: "high",
    4: "low",
    5: "close",
    6: "volume",})
        df = pd.DataFrame(df)


print(df.columns.tolist())
df

['Date', 'time', 'open', 'high', 'low', 'close', 'volume']


,Date,time,open,high,low,close,volume
0,2024.01.01,17:00,1.10427,1.10429,1.10425,1.10429,0
1,2024.01.01,17:01,1.10429,1.10429,1.10429,1.10429,0
2,2024.01.01,17:02,1.10429,1.10429,1.10429,1.10429,0
3,2024.01.01,17:03,1.10429,1.10429,1.10426,1.10426,0
4,2024.01.01,17:04,1.10426,1.10431,1.10424,1.10425,0
...,...,...,...,...,...,...,...
372374,2024.12.31,16:54,1.03554,1.03554,1.03545,1.03546,0
372375,2024.12.31,16:55,1.03546,1.03556,1.03546,1.03554,0
372376,2024.12.31,16:56,1.03552,1.03552,1.03537,1.03539,0
372377,2024.12.31,16:57,1.03540,1.03544,1.03539,1.03539,0


In [5]:
# Merge BID & ASK first
bid = dp.fetch(
    instrument="EUR/USD",
    interval=dp.INTERVAL_MIN_1,
    offer_side=dp.OFFER_SIDE_BID,
    start=datetime(2025, 7, 1),
    end=datetime.now()
)

ask = dp.fetch(
    instrument="EUR/USD",
    interval=dp.INTERVAL_MIN_1,
    offer_side=dp.OFFER_SIDE_ASK,
    start=datetime(2025, 7, 1),
    end=datetime.now()
)

df = bid.join(ask, lsuffix="_bid", rsuffix="_ask")

# Compute Mid-price OHLC
df["open"]  = (df["open_bid"]  + df["open_ask"]) / 2
df["high"]  = (df["high_bid"]  + df["high_ask"]) / 2
df["low"]   = (df["low_bid"]   + df["low_ask"]) / 2
df["close"] = (df["close_bid"] + df["close_ask"]) / 2

# Spread in pips (EURUSD → 4 decimals)
df["spread"] = (df["close_ask"] - df["close_bid"]) * 10000  

# Tick volume (take BID volume, or sum both sides if you prefer)
df["tick_volume"] = df["volume_bid"].round().astype(int)

# Real volume (Dukascopy doesn’t give it → 0)
df["real_volume"] = 0  

# Reset to MetaTrader-like schema
final = df.reset_index()[["timestamp", "open", "high", "low", "close", "tick_volume", "spread", "real_volume"]]
final = final.rename(columns={"timestamp": "time"})
final = pd.DataFrame(final)
final.head()


INFO:DUKASCRIPT:current timestamp :2025-07-29T21:09:00
INFO:DUKASCRIPT:current timestamp :2025-08-27T18:48:00
INFO:DUKASCRIPT:current timestamp :2025-09-05T21:59:00
INFO:DUKASCRIPT:current timestamp :2025-07-29T21:09:00
INFO:DUKASCRIPT:current timestamp :2025-08-27T18:48:00
INFO:DUKASCRIPT:current timestamp :2025-09-05T21:59:00


,time,open,high,low,close,tick_volume,spread,real_volume
0,2025-06-30 23:00:00+00:00,1.178590,1.17861,1.178495,1.178605,41,0.3,0
1,2025-06-30 23:01:00+00:00,1.178590,1.17863,1.178590,1.178605,18,0.5,0
2,2025-06-30 23:02:00+00:00,1.178600,1.17861,1.178550,1.178590,27,0.4,0
3,2025-06-30 23:03:00+00:00,1.178605,1.17865,1.178580,1.178590,51,0.6,0
4,2025-06-30 23:04:00+00:00,1.178580,1.17860,1.178575,1.178580,10,0.4,0


In [2]:
import dukascopy_python as dp

print(dir(dp))


['INTERVAL_DAY_1', 'INTERVAL_HOUR_1', 'INTERVAL_HOUR_4', 'INTERVAL_MIN_1', 'INTERVAL_MIN_10', 'INTERVAL_MIN_15', 'INTERVAL_MIN_30', 'INTERVAL_MIN_5', 'INTERVAL_MONTH_1', 'INTERVAL_SEC_1', 'INTERVAL_SEC_10', 'INTERVAL_SEC_30', 'INTERVAL_TICK', 'INTERVAL_WEEK_1', 'OFFER_SIDE_ASK', 'OFFER_SIDE_BID', 'TIME_UNIT_DAY', 'TIME_UNIT_HOUR', 'TIME_UNIT_MIN', 'TIME_UNIT_MONTH', 'TIME_UNIT_SEC', 'TIME_UNIT_TICK', 'TIME_UNIT_WEEK', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_fetch', '_get_custom_logger', '_get_dataframe_columns_for_timeunit', '_interval_units', '_resample_to_nearest', '_stream', 'datetime', 'fetch', 'json', 'live_fetch', 'logging', 'pd', 'random', 'requests', 'sleep', 'string', 'timedelta']
